In [1]:
!pip install datasets 
!pip install seqeval 
!pip install transformers[torch] -p
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2dcafb2c0ce6a4c9bf9a7e3ea30fa65b24189bd81830484bd8b8b46edb664ae4
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -p


In [2]:
import numpy as np
import pandas as pd
import wandb
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

2024-07-13 17:50:00.394868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 17:50:00.395009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 17:50:00.550905: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_path = "/kaggle/input/hackathon-online-aspect-based-sentiment-analysis/train.txt"
test_path = "/kaggle/input/hackathon-online-aspect-based-sentiment-analysis/test.csv"
sample_submission_path = "/kaggle/input/hackathon-online-aspect-based-sentiment-analysis/sample_submission.csv"

train = pd.read_csv(train_path, delimiter = "\t", header=None)
test = pd.read_csv(test_path, sep = ",", on_bad_lines='skip')
sample_submission = pd.read_csv(sample_submission_path)

In [4]:
test.columns = [0, 1]

In [5]:
from sklearn.model_selection import train_test_split

test_val = test
val_size = 0.2
test_val, val = train_test_split(test_val, test_size=val_size)

In [6]:
test_txt_path = '/kaggle/working/test.txt'
val_txt_path = '/kaggle/working/val.txt'

test.to_csv(test_txt_path, sep='\t', index=False,header=False)
val.to_csv(val_txt_path, sep='\t', index=False, header=True)

In [7]:
def load_data(fname, tag_id=None):
    data = []
    if tag_id is None:
        tag_id = {}

    with open(fname, encoding='utf-8') as fp:
        lines = fp.readlines()
        tmp_sen = []
        for line in lines:
            if line.strip() == "":
                if len(tmp_sen) > 0:
                    tokens = []
                    tags = []
                    for token, tag in tmp_sen:
                        tokens.append(token)
                        if tag not in tag_id.keys():
                            tag_id[tag] = len(tag_id)
                        tags.append(tag_id[tag])

                    data.append({"tokens": tokens, "tags": tags})
                    tmp_sen = []

                continue

            _x = line.strip().split('\t')
            if len(_x) == 2:  # Ensure that there are exactly two columns
                tmp_sen.append((_x[0], _x[1]))
            else:
                print(f"Skipping malformed line: {line.strip()}")

    if len(tmp_sen) > 0:  # In case the last sentence does not end with a blank line
        tokens = []
        tags = []
        for token, tag in tmp_sen:
            tokens.append(token)
            if tag not in tag_id.keys():
                tag_id[tag] = len(tag_id)
            tags.append(tag_id[tag])

        data.append({"tokens": tokens, "tags": tags})

    return data, tag_id

In [8]:
import json

def write_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json_line = json.dumps(entry)
            file.write(json_line + '\n')

In [9]:
test

,0,1
0,1_1,สินค้า
1,1_2,มี
2,1_3,การ
3,1_4,การกระแทก
4,1_5,อย่าง
...,...,...
6385,153_31,ความปลอดภัย
6386,153_32,ใน
6387,153_33,การใช้งาน
6388,153_34,ของ


In [10]:
tag_id = {'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

train_data, _ = load_data(train_path)
test_data,_ = load_data(test_txt_path,tag_id)
val_data,_ = load_data(val_txt_path,tag_id)

In [11]:
tag_id = {'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

In [12]:
tag_id

{'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

In [13]:
write_jsonl(train_data,'train.json')
write_jsonl(test_data,'test.json')
write_jsonl(val_data,'val.json')

In [14]:
task = "asp"
model_checkpoint = "Geotrend/bert-base-th-cased"
batch_size = 16

file_dict = {
  "train" : "./train.json",
  "test" : "./test.json",
  "val" : "./val.json"
}

datasets =  load_dataset(
  'json',
  data_files=file_dict
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [15]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 609
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1
    })
    val: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1
    })
})

In [16]:
datasets['train']['tags'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1]

In [17]:
tag_id

{'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

In [18]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tag_id))

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/370M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at Geotrend/bert-base-th-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/47.7k [00:00<?, ?B/s]

In [20]:
print(tokenizer("สวัสดีครับผมชื่อเอ"))
print(tokenizer(["สวัสดี","ครับ","ผม","ชื่อ", "เอ"]))

{'input_ids': [11, 175, 4953, 8342, 6910, 6602, 8305, 8473, 2939, 5972, 6398, 8484, 5136, 12], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[11, 175, 4953, 8342, 6910, 12], [11, 140, 3920, 8305, 12], [11, 162, 2939, 12], [11, 145, 6398, 12], [11, 184, 5136, 12]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]}


In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    label_all_tokens = True
    #print('tokenized_inputs:',tokenized_inputs)
    labels = []
    lab2index = {}
    word_idxs = []
    for i, label in enumerate(examples[f"tags"]):
        #print(label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        #print('x word_ids',word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)
        word_idxs.append(word_ids)
    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_idxs
    return tokenized_inputs

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/609 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['train']['tokens'][0])


In [38]:
tokenized_datasets['train']['input_ids'][0]
print(tokenized_datasets['train']['tokens'][0])
print(tokenized_datasets['train']['input_ids'][0])
print(tokenized_datasets['train']['tags'][0])
print(tokenized_datasets['train']['labels'][0])
print(tokenized_datasets['train']['word_ids'][0])

['สิ้น', 'ค้า', 'ครบถ้วน', 'ถูกต้อง', 'มา', 'ที่', 'สั่งซื้อ', 'ระยะเวลา', 'ขนส่งสินค้า', 'ไม่', 'ช้า', 'ไม่', 'เร็ว', 'กำลังดี', 'เหมาะสม', 'กับ', 'ราคา', 'ที่', 'เป็น', 'รอย', 'ดำ', 'เพราะ', 'ใส่', 'แล้ว', 'ไม่', 'โดน', 'กัด', 'คุ้มค่า', 'การขนส่ง', 'รวดเร็ว']
[11, 175, 4610, 6478, 140, 6018, 140, 3920, 6443, 8472, 5398, 7561, 156, 5671, 3167, 4799, 7652, 167, 2930, 6702, 175, 8480, 6104, 8071, 8482, 5398, 5136, 169, 3894, 3552, 3894, 8484, 4953, 7111, 139, 2617, 4665, 6104, 4665, 4358, 4949, 6018, 188, 2939, 5013, 145, 6018, 188, 2939, 5013, 184, 3920, 8490, 4953, 138, 6812, 3554, 6958, 6910, 184, 8477, 7322, 3894, 4665, 2939, 8085, 169, 2930, 4949, 2930, 6702, 7439, 169, 8261, 154, 6812, 184, 5795, 3920, 2930, 3894, 187, 4665, 5013, 185, 8092, 188, 2939, 5013, 186, 3827, 2617, 138, 7751, 140, 6729, 5398, 2939, 4949, 5724, 138, 6663, 7806, 2617, 4665, 6104, 169, 4953, 3827, 8484, 3920, 8490, 4953, 12]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [24]:
args = TrainingArguments(
    f"test-{task}",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
)

In [25]:
metric = load_metric("seqeval")

/tmp/ipykernel_33/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [26]:
id_tag = {value: key for key, value in tag_id.items()}
id_tag

{0: 'Appearance',
 1: 'DeliveryTime',
 2: 'Price',
 3: 'Quality',
 4: 'StoreService',
 5: 'NEG-Quality',
 6: 'O',
 7: 'NEG-Appearance',
 8: 'NEG-StoreService',
 9: 'Size',
 10: 'NEG-DeliveryTime',
 11: 'Packaging',
 12: 'NEG-Price',
 13: 'NEG-Size',
 14: 'NEG-Packaging'}

In [27]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id_tag[p].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_tag[l].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
wandb.login()
#87bcb981fbde472f632869f1bc5b2bfa8b11edeb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss


IndexError: Target 180 is out of bounds.

In [ ]:
trainer.evaluate()